In [1]:
# Import libraries and packages for the project 

from selenium import webdriver
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import random
import numpy as np
import pandas as pd
from underthesea import sentiment


print('- Finish importing packages')

- Finish importing packages


In [2]:
raw = pd.read_csv('ShopeeMall.csv')
raw.head()

,Order,Name,URL
0,1,Abbott,https://shopee.vn/abbottofficial
1,2,Adidas Official Store,https://shopee.vn/adidasofficialstore
2,3,Alobuy,https://shopee.vn/1khovn_bach_hoa_tien_loi
3,4,Amak,https://shopee.vn/nhasachamak
4,5,An Nguyên,https://shopee.vn/sachhayquanhieu


In [3]:
df = np.array(raw)

In [4]:
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver,20)
url = "https://shopee.vn/"
# Login to shopee account
# login_url = "https://shopee.vn/buyer/login/qr?next=https%3A%2F%2Fshopee.vn%2Fuser%2Fpurchase%2F"
# driver.get(login_url)
# sleep(40)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [/home/dijkstra/.wdm/drivers/chromedriver/linux64/104.0.5112.20/chromedriver] found in cache


In [5]:

headers = ['Order', 'Name', 'URL', "Rating","Chat Response (%)","Follower"]
char_dic = {'k':1000,'r': 1000000}
update_data = []

for shop in tqdm(df):

    try:
        driver.get(shop[2])
        print('- Accessing shop: ', shop[2])
        sleep(2)
        driver.execute_script('window.scrollTo(0, 0);') #scroll to the end of the page
        sleep(1)
    except TimeoutException:
        print("***************************")
        print("Loading took too much time!")
        print("***************************")
        continue


    loop_page_source = BeautifulSoup(driver.page_source,"html.parser")
    items = loop_page_source.find_all('div',{"class":["section-seller-overview__item-text-value"]})
    
    # Clean data
    if(len(items)==12):
        rating_raw = items[9].text
        chat_response_raw = items[4].text
        follower_raw = items[7].text.replace(",",".")
    else:
        rating_raw = items[11].text
        chat_response_raw = items[4].text
        follower_raw = items[9].text.replace(",",".")

    # Get rating of chat response 
    chat_response = chat_response_raw.split(" ")[0][:-1]

    # Get the number of follower
    unit_char = follower_raw[-1]
    if(unit_char.isdecimal()):
        follower = int(follower_raw[:-1])
    else:
        value = ""
        if(unit_char=="k"):
            value = follower_raw[:-1]
        else:
            value = follower_raw[:-2]
            
        follower = float(value)*char_dic[unit_char]

    # Get rating
    rating = rating_raw.split(" ")[0]


    update_data.append({headers[0]:shop[0],headers[1]:shop[1],headers[2]:shop[2],headers[3]:rating,headers[4]:chat_response,headers[5]:follower})

print("Finished crawling data!")


  0%|          | 0/100 [00:00<?, ?it/s]

- Accessing shop:  https://shopee.vn/abbottofficial
- Accessing shop:  https://shopee.vn/adidasofficialstore
- Accessing shop:  https://shopee.vn/1khovn_bach_hoa_tien_loi
- Accessing shop:  https://shopee.vn/nhasachamak
- Accessing shop:  https://shopee.vn/sachhayquanhieu
- Accessing shop:  https://shopee.vn/anbertmangashop
- Accessing shop:  https://shopee.vn/anbertmangashop3103
- Accessing shop:  https://shopee.vn/anessaofficialstore
- Accessing shop:  https://shopee.vn/ankervn
- Accessing shop:  https://shopee.vn/anlene_officialstore
- Accessing shop:  https://shopee.vn/babyhop_store
- Accessing shop:  https://shopee.vn/balocongnghe
- Accessing shop:  https://shopee.vn/bambamstore274
- Accessing shop:  https://shopee.vn/hieusachbanhmi
- Accessing shop:  https://shopee.vn/banxeer.vn
- Accessing shop:  https://shopee.vn/baongocjewelry
- Accessing shop:  https://shopee.vn/baseus.official.mall.vn
- Accessing shop:  https://shopee.vn/basike.vn
- Accessing shop:  https://shopee.vn/choviet

In [6]:
with open('ShopeeMall5.csv', 'w',  newline = '') as file_output:
    
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for shop in update_data:
        writer.writerow(shop)

print('Mission Completed!')

Mission Completed!
